In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_mod = pd.read_csv('../data/demo_cluster_data.csv')

In [3]:
df_mod.head()

,total_submitted_charges_norm,Adult obesity_% Obese,Physical inactivity_% Physically Inactive,Poor mental health days_Mentally Unhealthy Days,Uninsured_% Uninsured,Primary Care_PCP Ratio,Rec Facility Rate_Rec Facility Rate,Limited acess to healthy foods_% Limited Access,% Fast Foods_% Fast Foods,predictions,county
0,560.764619,34.1,32.2,4.3,13.6,2732,7.321046,9.793530,52.173913,0,Autauga
1,719.903586,25.5,25.1,3.8,19.1,1347,8.730813,5.460261,35.443038,0,Baldwin
2,621.280455,37.0,34.2,4.1,18.5,2283,3.650968,11.420316,55.882353,0,Barbour
3,236.284176,34.0,37.2,5.3,17.7,3813,4.370629,2.144661,45.454545,0,Bibb
4,516.343006,32.0,35.9,4.0,19.3,4782,5.227665,1.062468,56.410256,0,Blount


In [9]:
df_mod.columns = ['submitted_charges_norm', 'obesity', 'inactivity', 'poor_mental_health', 
                  'uninsured', 'primary_care', 'rec_facility', 'limited_access_health_foods', 
                  'fast_food', 'clusters', 'county']

In [21]:
df_mod['cluster_1'] = [1 if x==1 else 0 for x in df_mod['clusters']]

In [22]:
df_mod.columns

Index([u'submitted_charges_norm', u'obesity', u'inactivity',
       u'poor_mental_health', u'uninsured', u'primary_care', u'rec_facility',
       u'limited_access_health_foods', u'fast_food', u'clusters', u'county',
       u'cluster_1'],
      dtype='object')

In [23]:
np.sum(df_mod['cluster_1'])

816

In [12]:
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2810 entries, 0 to 2809
Data columns (total 12 columns):
submitted_charges_norm         2810 non-null float64
obesity                        2810 non-null float64
inactivity                     2810 non-null float64
poor_mental_health             2628 non-null float64
uninsured                      2810 non-null float64
primary_care                   2765 non-null float64
rec_facility                   2810 non-null float64
limited_access_health_foods    2810 non-null float64
fast_food                      2763 non-null float64
clusters                       2810 non-null int64
county                         2810 non-null object
cluster_1                      2810 non-null int64
dtypes: float64(9), int64(2), object(1)
memory usage: 285.4+ KB


Need to fill null values.

In [24]:
df_mod['poor_mental_health'].fillna(np.mean(df_mod['poor_mental_health']), inplace=True)

In [25]:
df_mod['primary_care'].fillna(np.mean(df_mod['primary_care']), inplace=True)

In [26]:
df_mod['fast_food'].fillna(np.mean(df_mod['fast_food']), inplace=True)

## Random Forest

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

In [27]:
y = df_mod.pop('cluster_1')

In [28]:
X = df_mod.iloc[:,1:9].values

In [29]:
X

array([[ 34.1       ,  32.2       ,   4.3       , ...,   7.32104618,
          9.7935304 ,  52.17391304],
       [ 25.5       ,  25.1       ,   3.8       , ...,   8.73081267,
          5.46026062,  35.44303798],
       [ 37.        ,  34.2       ,   4.1       , ...,   3.65096751,
         11.42031572,  55.88235294],
       ..., 
       [ 30.9       ,  23.1       ,   3.4       , ...,   9.47104229,
          7.19014955,  43.24324324],
       [ 24.5       ,  25.3       ,   2.3       , ...,  23.38634238,
          2.73793925,  36.36363636],
       [ 28.8       ,  28.2       ,   3.        , ...,  13.93145723,
          3.98922185,  20.        ]])

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [61]:
rf_mod = RandomForestClassifier(n_estimators=100)

In [62]:
rf_mod.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [63]:
roc_auc_score(y_test, rf_mod.predict_proba(X_test)[:,1])

0.79953145917001334

In [64]:
zip(df_mod.columns[1:9], rf_mod.feature_importances_)

[('obesity', 0.10930915456197948),
 ('inactivity', 0.12057099625435186),
 ('poor_mental_health', 0.11932320526667267),
 ('uninsured', 0.12615093448004319),
 ('primary_care', 0.14054878754474204),
 ('rec_facility', 0.10264229792413093),
 ('limited_access_health_foods', 0.11916260723237335),
 ('fast_food', 0.16229201673570653)]

In [66]:
rf_mod.score(X_test, y_test)

0.80782918149466187